## Import required libraries and setup dataframes

In [1]:
sc.install_pypi_package("s3fs")
sc.install_pypi_package("pandas")
import pandas as pd
import numpy as np
import warnings
from s3fs import S3FileSystem

# Setup max column number to 100 to avoid truncating movieLens dataframe.
# pd.set_option('print.max_columns', None)
# pd.set_option('print.max_rows', 10000)
# pd.set_option('print.max_colwidth', None)

warnings.simplefilter('ignore', category = FutureWarning)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1599465122176_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/92/15/792d40668bd63acc8f816188ed3c3a27f393717122c8bed5fe8f550616da/s3fs-0.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/a6/3d032b308fa546a5223daf0ae5e414af3ddea33678fff2834d99bb0dd909/aiobotocore-1.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d3/66/974e01194980d9780cc09724315111f9cccba26b4351552fdb4d97eb842e/fsspec-0.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/83/42/90df27c516ce54fa26964bc4a632ecaf352c7e99574b515255e48b4a7cc7/aioitertools-0.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1e/6a/b6490235c01c941a24a86235e2a641e9505cf0ce4b4968d4987573d92bec/botocore-1.17.44-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/71/6000eacb8923d9fd07aa8784a8fab4f022ae697f3c2456d7dca75c743dd6/aiohttp-3.6.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages

## Data clean-up

Get original dataset _movies.dat_ from disc of local machine.

Data downloaded from https://grouplens.org/datasets/movielens/25m/

In [2]:
movieLensDatasetBucketPath = "s3://movielens-dataset-bucket/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
movieLensDatasetPath = movieLensDatasetBucketPath + "movies.dat"
df_movieLens = pd.read_table(movieLensDatasetPath, \
                             # in the original dataset, fields are separated by '@'
                             sep = '@', \
                             # don't get original headers
                             header = None, \
                             # name columns
                             names = ['ID', 'Title', 'Genres'], \
                             # setup dataframe's index
                             index_col = 'ID')

# display first 10 rows of dataset
df_movieLens.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                 Title                                       Genres
ID                                                                                 
1                     Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
2                       Jumanji (1995)                   Adventure|Children|Fantasy
3              Grumpier Old Men (1995)                               Comedy|Romance
4             Waiting to Exhale (1995)                         Comedy|Drama|Romance
5   Father of the Bride Part II (1995)                                       Comedy
6                          Heat (1995)                        Action|Crime|Thriller
7                       Sabrina (1995)                               Comedy|Romance
8                  Tom and Huck (1995)                           Adventure|Children
9                  Sudden Death (1995)                                       Action
10                    GoldenEye (1995)                    Action|Adventure|T

Dataset was loaded correctly to df_movieLens variable.

As we can see, there are 3 colums:
* ID - unique movie's identificator
* Title - movie's title and relese year
* Genres - movies's classification

Columns: 'Title' and 'Genres' will be splitted for further analysis.

Get original dataset _ratings.dat_ from disc of local machine.

Data downloaded from https://grouplens.org/datasets/movielens/25m/

In [4]:
movieLensRatesDatasetPath = movieLensDatasetBucketPath + "ratings.dat"
df_movieLensRates = pd.read_table(movieLensRatesDatasetPath, \
                             # in the original dataset, fields are separated by '@'
                             sep = '@', \
                             
                             # don't get original headers
                             header = None, \
                             
                             # name columns
                             names = ['ID', 'UserID', 'Rate', 'Timestamp'], \
                             
                             # setup dataframe's index
                             index_col = 'ID')

# display first 10 rows of dataset
df_movieLensRates.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    UserID  Rate  Timestamp
ID                         
1      122   5.0  838985046
1      185   5.0  838983525
1      231   5.0  838983392
1      292   5.0  838983421
1      316   5.0  838983392
1      329   5.0  838983392
1      355   5.0  838984474
1      356   5.0  838983653
1      362   5.0  838984885
1      364   5.0  838983707

Dataset was loaded correctly to _df_movieLensRates_ variable.

As we can see, there are 4 colums:
* ID - unique movie's identificator
* UserID - unique user's identificator
* Rate - ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars)
* Timestamp - timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970

Column 'Timestamp' will be converted to date.

Having loaded datasets, let's check whether there are empty values.

In [5]:
print(df_movieLens.isnull().sum())
print(df_movieLensRates.isnull().sum())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Title     0
Genres    0
dtype: int64
UserID       0
Rate         0
Timestamp    0
dtype: int64

Fortunately there are no empty values. Datasets don't require filling or deleting values.

Column Title contains both title and relese year of a movie. This should be splitted into two columns: 'Title', 'Year'.

In [6]:
# assigne release year for each movie to Pandas Series
year = pd.Series(df_movieLens['Title']).str[-5:-1].astype('int')
year.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ID
1    1995
2    1995
3    1995
4    1995
5    1995
Name: Title, dtype: int64

In [7]:
# assigne title for each movie to Pandas Series
title = pd.Series(df_movieLens['Title']).str[:-7].astype('str')
title.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ID
1                      Toy Story
2                        Jumanji
3               Grumpier Old Men
4              Waiting to Exhale
5    Father of the Bride Part II
Name: Title, dtype: object

In [8]:
# overwrite 'Title' column with single movie's title
df_movieLens['Title'] = title

# add 'Year' column extracted earlier from original 'Title' column
df_movieLens['Year'] = year

df_movieLens.head(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                             Title  ...  Year
ID                                  ...      
1                        Toy Story  ...  1995
2                          Jumanji  ...  1995
3                 Grumpier Old Men  ...  1995
4                Waiting to Exhale  ...  1995
5      Father of the Bride Part II  ...  1995
6                             Heat  ...  1995
7                          Sabrina  ...  1995
8                     Tom and Huck  ...  1995
9                     Sudden Death  ...  1995
10                       GoldenEye  ...  1995
11         American President, The  ...  1995
12     Dracula: Dead and Loving It  ...  1995
13                           Balto  ...  1995
14                           Nixon  ...  1995
15                Cutthroat Island  ...  1995
16                          Casino  ...  1995
17           Sense and Sensibility  ...  1995
18                      Four Rooms  ...  1995
19  Ace Ventura: When Nature Calls  ...  1995
20                     Money Train

Value in 'Genres' column in dataset _movies.dat_ is a list of genres assigned to each movie. Movie can be classified to more than one genre thus genres are separated by pipe '|'. In that form, it's not possible to analyse dataset.

Let's see how splitting would work.

In [9]:
pd.Series(df_movieLens['Genres']).str.split('|', expand = True).head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            0          1         2       3        4     5     6     7
ID                                                                   
1   Adventure  Animation  Children  Comedy  Fantasy  None  None  None
2   Adventure   Children   Fantasy    None     None  None  None  None
3      Comedy    Romance      None    None     None  None  None  None
4      Comedy      Drama   Romance    None     None  None  None  None
5      Comedy       None      None    None     None  None  None  None

Based on first 5 rows of dataset, we can see that simply splitting is not right solution. List of genres for each movie was splitted into single genre however the form of table is not ready to analysis at all.

Here, we would like to have each genre as a new column with values either 1 (if movie is classified) or 0 (if movie is not classified). Example for the first row from table above:

|ID|Adventure|Animation|Children|Comedy|Fantasy|Romance|Drama|Horror|etc...|
|-|-|-|-|-|-|-|-|-|-|
|1|1|1|1|1|1|0|0|0|etc...|
|2|1|0|1|0|1|0|0|0|etc...|

Before it would be able to create new columns for each genre, distinct genres should be found.

In [10]:
# assigne unique genres to list
uniqueValues = pd.DataFrame(df_movieLens['Genres'].str.split('|').tolist()).stack().unique()
uniqueValues

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Film-Noir', 'Western', '(no genres listed)'], dtype=object)

In [11]:
# add genres columns and fill with 0 and 1
df_movieLens = df_movieLens.join(df_movieLens['Genres'].str.get_dummies())

# # drop 'Genres' column
df_movieLens.drop('Genres', inplace = True, axis = 1)

# display first 10 rows of dataset
df_movieLens.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                          Title  Year  ...  War  Western
ID                                     ...              
1                     Toy Story  1995  ...    0        0
2                       Jumanji  1995  ...    0        0
3              Grumpier Old Men  1995  ...    0        0
4             Waiting to Exhale  1995  ...    0        0
5   Father of the Bride Part II  1995  ...    0        0
6                          Heat  1995  ...    0        0
7                       Sabrina  1995  ...    0        0
8                  Tom and Huck  1995  ...    0        0
9                  Sudden Death  1995  ...    0        0
10                    GoldenEye  1995  ...    0        0

[10 rows x 22 columns]

Converting 'Timestamp' column in ratings dataframe.

In [12]:
# check what type are values in 'Timestamp' column
df_movieLensRates.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UserID         int64
Rate         float64
Timestamp      int64
dtype: object

In [13]:
# 'Timestamp' is integer and values are seconds from midnight epoch time, so it is possible to use [unit = 's'] parameter
df_movieLensRates['Timestamp'] = pd.to_datetime(df_movieLensRates['Timestamp'], unit= 's')

# display first 5 rows of dataset
print(df_movieLensRates.head())

# display last 5 rows of dataset
print(df_movieLensRates.tail())

# display random 15 rows of dataset
print(df_movieLensRates.sample(n = 10))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    UserID  Rate           Timestamp
ID                                  
1      122   5.0 1996-08-02 11:24:06
1      185   5.0 1996-08-02 10:58:45
1      231   5.0 1996-08-02 10:56:32
1      292   5.0 1996-08-02 10:57:01
1      316   5.0 1996-08-02 10:56:32
       UserID  Rate           Timestamp
ID                                     
71567    2107   1.0 1998-12-02 06:35:53
71567    2126   2.0 1998-12-03 01:39:03
71567    2294   5.0 1998-12-02 05:52:48
71567    2338   2.0 1998-12-02 05:53:36
71567    2384   2.0 1998-12-02 05:56:13
       UserID  Rate           Timestamp
ID                                     
54970     141   4.5 2006-08-09 04:19:45
23189    3977   3.5 2005-05-05 19:30:41
17153    1409   3.0 1997-04-02 23:39:03
28568    2321   3.0 1999-10-21 07:38:18
63071     597   3.0 2000-05-07 21:09:40
51882    2789   0.5 2007-06-02 03:17:58
18652     858   4.5 2006-02-16 00:51:43
33547    6708   4.0 2003-10-24 17:57:40
48815    4007   3.0 2005-02-02 05:01:11
34572    2572   3.0 2

Datasets:
* movies.dat -> df_movieLens
* rating.dat -> df_movieLensRates

were cleaned-up and prepared for further analysis.

Here, I'll save joined datasets to csv to use it later for analysis in Spark.

In [14]:
df_moviesAndRates = df_movieLens.join(df_movieLensRates, how = 'inner')
df_moviesAndRates

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         Title  Year  ...  Rate           Timestamp
ID                                    ...                          
1                    Toy Story  1995  ...   5.0 1996-08-02 11:24:06
1                    Toy Story  1995  ...   5.0 1996-08-02 10:58:45
1                    Toy Story  1995  ...   5.0 1996-08-02 10:56:32
1                    Toy Story  1995  ...   5.0 1996-08-02 10:57:01
1                    Toy Story  1995  ...   5.0 1996-08-02 10:56:32
...                        ...   ...  ...   ...                 ...
65133  Blackadder Back & Forth  1999  ...   3.0 2000-05-12 17:39:16
65133  Blackadder Back & Forth  1999  ...   2.0 2000-05-12 17:37:52
65133  Blackadder Back & Forth  1999  ...   3.0 2000-05-12 17:37:37
65133  Blackadder Back & Forth  1999  ...   4.0 2000-05-12 17:37:15
65133  Blackadder Back & Forth  1999  ...   3.0 2000-05-12 17:38:26

[1462309 rows x 25 columns]

#### Bonus!

It was found that movies with 'The' prefix is displayed in weird form, eg. 'American President, The' instead of 'The American President' or French 'Père Noël est une Ordure, Le' instead of 'Le Père Noël est une Ordure'.
There are also titles with comma, so this will also be updated to use dot, eg." 'Vernon, Florida' -> 'Vernon. Florida'
Let's correct it as a bonus task.

As the example I'll use movie 'The American President', 'Le Père Noël est une Ordure' and 'Vernon. Florida'.

In [15]:
# reset index to find movie 'American President, The' - this movie will be my example to show replacement process
df_moviesAndRates_v1 = df_moviesAndRates.reset_index()

# display all records of movie 'American President'
df_moviesAndRates_v1.loc[(df_moviesAndRates_v1['Title'] == 'American President, The') | (df_moviesAndRates_v1['Title'] == 'Père Noël est une Ordure, Le') | (df_moviesAndRates_v1['Title'] == 'Vernon, Florida')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          ID                         Title  ...  Rate           Timestamp
1298      11       American President, The  ...   5.0 1999-12-22 15:40:15
1299      11       American President, The  ...   3.0 1999-12-24 05:08:25
1300      11       American President, The  ...   5.0 1999-12-22 15:44:55
1301      11       American President, The  ...   5.0 1999-12-22 15:31:08
1302      11       American President, The  ...   4.0 1999-12-22 15:30:39
...      ...                           ...  ...   ...                 ...
798337  6119  Père Noël est une Ordure, Le  ...   2.0 2008-11-23 17:19:27
798338  6119  Père Noël est une Ordure, Le  ...   2.0 2008-11-06 10:57:45
798339  6119  Père Noël est une Ordure, Le  ...   3.5 2008-07-27 20:57:20
798340  6119  Père Noël est une Ordure, Le  ...   3.0 2008-10-29 13:52:28
798341  6119  Père Noël est une Ordure, Le  ...   2.5 2008-09-29 14:49:55

[499 rows x 26 columns]

For replacement, I'll use _Series.str.replace_ function.

Here, there are two solutions. I'll present them both:
   1. simply delete word 'The' and 'Le' at the end of each movie's title
   2. move word 'The', 'Le' and replacing ',' to '.' from the end to beginning of title

1. Delete word 'The' and 'Le' at the end of each movie's title.

In [16]:
# delete prefix 'The' from the end of title
df_moviesAndRates_v1['Title'] = df_moviesAndRates_v1['Title'].str.replace(', The', '', case = True)
df_moviesAndRates_v1['Title'] = df_moviesAndRates_v1['Title'].str.replace(',The', '', case = True)
df_moviesAndRates_v1['Title'] = df_moviesAndRates_v1['Title'].str.replace(', Le', '', case = True)
df_moviesAndRates_v1['Title'] = df_moviesAndRates_v1['Title'].str.replace(', ', ' ', case = True)
df_moviesAndRates_v1['Title'] = df_moviesAndRates_v1['Title'].str.replace(',', '.', case = True)

# display all records of movie 'The Temp'
df_moviesAndRates_v1.loc[(df_moviesAndRates_v1['Title'] == 'American President') | (df_moviesAndRates_v1['Title'] == 'Père Noël est une Ordure') | (df_moviesAndRates_v1['Title'] == 'Vernon Florida')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          ID                     Title  Year  ...  UserID  Rate           Timestamp
1298      11        American President  1995  ...     260   5.0 1999-12-22 15:40:15
1299      11        American President  1995  ...     296   3.0 1999-12-24 05:08:25
1300      11        American President  1995  ...     349   5.0 1999-12-22 15:44:55
1301      11        American President  1995  ...     440   5.0 1999-12-22 15:31:08
1302      11        American President  1995  ...     480   4.0 1999-12-22 15:30:39
...      ...                       ...   ...  ...     ...   ...                 ...
798337  6119  Père Noël est une Ordure  1982  ...   60037   2.0 2008-11-23 17:19:27
798338  6119  Père Noël est une Ordure  1982  ...   60040   2.0 2008-11-06 10:57:45
798339  6119  Père Noël est une Ordure  1982  ...   60074   3.5 2008-07-27 20:57:20
798340  6119  Père Noël est une Ordure  1982  ...   61132   3.0 2008-10-29 13:52:28
798341  6119  Père Noël est une Ordure  1982  ...   61350   2.5 2008-09-29 1

In [17]:
df_moviesAndRates_v1.loc[df_moviesAndRates_v1['Title'] == 'Vernon Florida']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          ID           Title  Year  ...  UserID  Rate           Timestamp
766684  5914  Vernon Florida  1981  ...      88   3.5 2008-07-25 22:51:16
766685  5914  Vernon Florida  1981  ...     260   5.0 2008-07-25 23:30:54
766686  5914  Vernon Florida  1981  ...     916   4.5 2008-07-25 22:50:52
766687  5914  Vernon Florida  1981  ...    1021   3.0 2008-07-25 23:28:04
766688  5914  Vernon Florida  1981  ...    1096   4.0 2008-07-25 22:52:16
766689  5914  Vernon Florida  1981  ...    1196   5.0 2008-07-25 23:30:34
766690  5914  Vernon Florida  1981  ...    1210   4.0 2008-07-25 23:30:49
766691  5914  Vernon Florida  1981  ...    1693   3.5 2008-07-25 22:51:13
766692  5914  Vernon Florida  1981  ...    2010   5.0 2008-07-25 22:51:28
766693  5914  Vernon Florida  1981  ...    2394   3.0 2008-07-25 22:50:25
766694  5914  Vernon Florida  1981  ...    2413   4.0 2008-07-25 22:51:58
766695  5914  Vernon Florida  1981  ...    2530   4.0 2008-07-25 23:29:05
766696  5914  Vernon Florida  1981  ..

2. Move word 'The', 'Le' and replacing ',' to '.' from the end to beginning of title.

In [18]:
# reset index to find movie 'American President, The' - this movie will be my example to show replacement process
df_moviesAndRates_v2 = df_moviesAndRates.reset_index()

# display 10 records of movie 'The American President'
df_moviesAndRates_v2.loc[(df_moviesAndRates_v2['Title'] == 'American President, The') | (df_moviesAndRates_v2['Title'] == 'Père Noël est une Ordure, Le')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          ID                         Title  ...  Rate           Timestamp
1298      11       American President, The  ...   5.0 1999-12-22 15:40:15
1299      11       American President, The  ...   3.0 1999-12-24 05:08:25
1300      11       American President, The  ...   5.0 1999-12-22 15:44:55
1301      11       American President, The  ...   5.0 1999-12-22 15:31:08
1302      11       American President, The  ...   4.0 1999-12-22 15:30:39
...      ...                           ...  ...   ...                 ...
798337  6119  Père Noël est une Ordure, Le  ...   2.0 2008-11-23 17:19:27
798338  6119  Père Noël est une Ordure, Le  ...   2.0 2008-11-06 10:57:45
798339  6119  Père Noël est une Ordure, Le  ...   3.5 2008-07-27 20:57:20
798340  6119  Père Noël est une Ordure, Le  ...   3.0 2008-10-29 13:52:28
798341  6119  Père Noël est une Ordure, Le  ...   2.5 2008-09-29 14:49:55

[472 rows x 26 columns]

In [19]:
# move prefix 'The' from the end to beginning of title using lambda function
df_moviesAndRates_v2['Title'] = df_moviesAndRates_v2['Title'].apply(lambda x: 'The ' + x.split(',')[0] if ', The' in x else x)
df_moviesAndRates_v2['Title'] = df_moviesAndRates_v2['Title'].apply(lambda x: 'The ' + x.split(',')[0] if ',The' in x else x)
df_moviesAndRates_v2['Title'] = df_moviesAndRates_v2['Title'].apply(lambda x: 'Le ' + x.split(',')[0] if ', Le' in x else x)
df_moviesAndRates_v2['Title'] = df_moviesAndRates_v2['Title'].str.replace(', ', '. ', case = True)
df_moviesAndRates_v2['Title'] = df_moviesAndRates_v2['Title'].str.replace(',', '.', case = True)

# display 10 records of movie 'The American President'
df_moviesAndRates_v2.loc[(df_moviesAndRates_v2['Title'] == 'The American President') | (df_moviesAndRates_v2['Title'] == 'Le Père Noël est une Ordure') | (df_moviesAndRates_v2['Title'] == 'Vernon. Florida')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          ID                        Title  ...  Rate           Timestamp
1298      11       The American President  ...   5.0 1999-12-22 15:40:15
1299      11       The American President  ...   3.0 1999-12-24 05:08:25
1300      11       The American President  ...   5.0 1999-12-22 15:44:55
1301      11       The American President  ...   5.0 1999-12-22 15:31:08
1302      11       The American President  ...   4.0 1999-12-22 15:30:39
...      ...                          ...  ...   ...                 ...
798337  6119  Le Père Noël est une Ordure  ...   2.0 2008-11-23 17:19:27
798338  6119  Le Père Noël est une Ordure  ...   2.0 2008-11-06 10:57:45
798339  6119  Le Père Noël est une Ordure  ...   3.5 2008-07-27 20:57:20
798340  6119  Le Père Noël est une Ordure  ...   3.0 2008-10-29 13:52:28
798341  6119  Le Père Noël est une Ordure  ...   2.5 2008-09-29 14:49:55

[499 rows x 26 columns]

Our final table is stored in variable df_moviesAndRates_v2. Let's save the output to .csv.

In [20]:
df_moviesAndRates_v2.to_csv('s3://movielens-dataset-bucket/moviesAndRates.csv', index = False)

print('Dataset is saved!')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dataset is saved!

## List of distinct genres of movies

List of distinct genres was created earlier during data clean-up thus to repeat this, there is need to get original data once again.

In [21]:
df_movieLensDistinctGenres = pd.read_table(movieLensDatasetPath, \
                             # in the original dataset, fields are separated by '@'
                             sep = '@', \
                             # don't get original headers
                             header = None, \
                             # name columns
                             names = ['ID', 'Title', 'Genres'], \
                             # setup dataframe's index
                             index_col = 'ID')

# str.split('|') -> split genres separated by pipe '|'
# tolist() -> save genres to list
# stack() -> reshape Series
# unique() -> get distinct values
distinctGenres = pd.DataFrame(df_movieLensDistinctGenres['Genres'].str.split('|').tolist()).stack().unique()

# display and check how many distinct genres are in the dataset
print(distinctGenres)
print(f'There are: {len(distinctGenres)} unique movie genres.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Adventure' 'Animation' 'Children' 'Comedy' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Mystery' 'Sci-Fi' 'IMAX'
 'Documentary' 'War' 'Musical' 'Film-Noir' 'Western' '(no genres listed)']
There are: 20 unique movie genres.

---
Before I started getting more experienced with Pandas magic functions, I had found solution using Python's 'regular' syntax.

Below, there is function which takes two arguments:
* Pandas Series (column of DataFrame)
* separator

In [22]:
df_movieLensDistinctGenresFunction = pd.read_table(movieLensDatasetPath, \
                             # in the original dataset, fields are separated by '@'
                             sep = '@', \
                             # don't get original headers
                             header = None, \
                             # name columns
                             names = ['ID', 'Title', 'Genres'], \
                             # setup dataframe's index
                             index_col = 'ID')

# definition of function
def uniqueValues(dataframeColumn, separator):
    uniqueGenres = pd.Series(dataframeColumn).unique()
    split = [genre.split(separator) for genre in uniqueGenres]

    splitList = []
    for genres in split:
        for genre in genres:
            splitList.append(genre)
            uniqueGenresSet = set(splitList)
    return list(uniqueGenresSet)

# call uniqueValues function with required arguments: DataFrame column and separator
distinctGenresFunction = uniqueValues(df_movieLensDistinctGenresFunction['Genres'], '|')

# display and check how many distinct genres are in the dataset
print(distinctGenresFunction)
print(f'There are: {len(distinctGenresFunction)} unique movie genres.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Sci-Fi', 'Adventure', 'Romance', 'Drama', 'Animation', 'Western', 'Documentary', 'Thriller', 'Fantasy', 'War', 'Film-Noir', 'Horror', 'Action', '(no genres listed)', 'Musical', 'Children', 'Crime', 'Comedy', 'Mystery', 'IMAX']
There are: 20 unique movie genres.

## Standard deviation of rate for each movie

Before standard deviation will be calculated, I'm goin to join 'Title' from movies dataset to rating dataset. This will allow to see standard deviation for movies rather than theirs IDs.

In [23]:
df_moviesAndRates = df_movieLensRates.join(df_movieLens['Title']) # could be how = 'inner' but I want to show dropna() function.

# 'Timestamp' column is not needed so it can be dropped
df_moviesAndRates.drop('Timestamp', inplace = True, axis = 1)
print(df_moviesAndRates.head())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    UserID  Rate      Title
ID                         
1      122   5.0  Toy Story
1      185   5.0  Toy Story
1      231   5.0  Toy Story
1      292   5.0  Toy Story
1      316   5.0  Toy Story

Default Pandas join is LEFT, so it could be possible that there are more rates in ratings.dat than movies in movies.dat.

**I could use how = 'inner'** but I wanted to show dropna() function.

After joining tables let's check whether there are empty values.

In [24]:
print(df_moviesAndRates.isnull().sum())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UserID          0
Rate            0
Title     8537745
dtype: int64

Based on results above, there are **missing 8537745 movies in movies.dat** vs. ratings.dat

.. that's a lot but once the assumption is to analyze standard deviation per movie's title, I'll drop empty values.

In [25]:
df_moviesAndRates = df_moviesAndRates.dropna()
df_moviesAndRates

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UserID  Rate                    Title
ID                                          
1         122   5.0                Toy Story
1         185   5.0                Toy Story
1         231   5.0                Toy Story
1         292   5.0                Toy Story
1         316   5.0                Toy Story
...       ...   ...                      ...
65133    3555   3.0  Blackadder Back & Forth
65133    3557   2.0  Blackadder Back & Forth
65133    3576   3.0  Blackadder Back & Forth
65133    3578   4.0  Blackadder Back & Forth
65133    3608   3.0  Blackadder Back & Forth

[1462309 rows x 3 columns]

In [26]:
print(df_moviesAndRates.isnull().sum())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UserID    0
Rate      0
Title     0
dtype: int64

Thanks for using _dropna()_ all empty values have been deleted.

To display standard deviation for title, first there is need to reset index and use 'Title' as index (current index is 'ID').

In [27]:
df_moviesAndRates = df_moviesAndRates.reset_index().set_index('Title')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, dataset is ready to calculate standard deviation.

Let's display 5 movies with the greatest standard deviation and 5 movies with the smallest one.

In [28]:
print(df_moviesAndRates['Rate'].groupby(['Title']).std().nlargest(5))

print(df_moviesAndRates['Rate'].groupby(['Title']).std().nsmallest(5))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Title
Taxi to the Dark Side            2.296880
U.S. Marshals                    1.990004
Sherlock Holmes in Washington    1.984823
Temp, The                        1.912539
August Evening                   1.899446
Name: Rate, dtype: float64
Title
Ballad of the Sad Cafe, The    0.0
From Beyond                    0.0
Hardball                       0.0
Layer Cake                     0.0
Toy Story                      0.0
Name: Rate, dtype: float64

**High standard deviation**

High standard deviation indicates that the values are distributed over a wide range.
The highest standard deviation is _2.296880_ for movie _Taxi to the Dark Side_.

**Low standard deviation**

Low standard deviation indicates that the values tend to be close to the mean. However if standard deviation equals to 0 means that all values are equal to mean and this must mean that all valeus are the same.
This is result of situation when all users rate movie the same. Example:

In [29]:
df_ToyStoryRates = df_moviesAndRates.loc[df_moviesAndRates['ID'] == 1]

print(f"\nAverage rate is {df_ToyStoryRates['Rate'].mean()}")
print(df_ToyStoryRates)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Average rate is 5.0
           ID  UserID  Rate
Title                      
Toy Story   1     122   5.0
Toy Story   1     185   5.0
Toy Story   1     231   5.0
Toy Story   1     292   5.0
Toy Story   1     316   5.0
Toy Story   1     329   5.0
Toy Story   1     355   5.0
Toy Story   1     356   5.0
Toy Story   1     362   5.0
Toy Story   1     364   5.0
Toy Story   1     370   5.0
Toy Story   1     377   5.0
Toy Story   1     420   5.0
Toy Story   1     466   5.0
Toy Story   1     480   5.0
Toy Story   1     520   5.0
Toy Story   1     539   5.0
Toy Story   1     586   5.0
Toy Story   1     588   5.0
Toy Story   1     589   5.0
Toy Story   1     594   5.0
Toy Story   1     616   5.0

As shown above for _Toy Story_ all rates are equal to 5 thus mean equals to 5 thus standard deviation equals to 0.

## Correlation between number of rates and average rate

In [30]:
df_correlationNumrateAvgrate = df_moviesAndRates.reset_index().set_index('ID')
df_correlationNumrateAvgrate.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        Title  UserID  Rate
ID                         
1   Toy Story     122   5.0
1   Toy Story     185   5.0

In [31]:
df_correlationNumrateAvgrate['NumRate'] = df_correlationNumrateAvgrate['Rate'].groupby('ID').count().astype('int')
df_correlationNumrateAvgrate

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         Title  UserID  Rate  NumRate
ID                                                   
1                    Toy Story     122   5.0       22
1                    Toy Story     185   5.0       22
1                    Toy Story     231   5.0       22
1                    Toy Story     292   5.0       22
1                    Toy Story     316   5.0       22
...                        ...     ...   ...      ...
65133  Blackadder Back & Forth    3555   3.0       27
65133  Blackadder Back & Forth    3557   2.0       27
65133  Blackadder Back & Forth    3576   3.0       27
65133  Blackadder Back & Forth    3578   4.0       27
65133  Blackadder Back & Forth    3608   3.0       27

[1462309 rows x 4 columns]

In [32]:
df_correlationNumrateAvgrate['AvgRate'] = df_correlationNumrateAvgrate['Rate'].groupby('ID').mean().astype('float')
df_correlationNumrateAvgrate

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         Title  UserID  Rate  NumRate   AvgRate
ID                                                             
1                    Toy Story     122   5.0       22  5.000000
1                    Toy Story     185   5.0       22  5.000000
1                    Toy Story     231   5.0       22  5.000000
1                    Toy Story     292   5.0       22  5.000000
1                    Toy Story     316   5.0       22  5.000000
...                        ...     ...   ...      ...       ...
65133  Blackadder Back & Forth    3555   3.0       27  3.518519
65133  Blackadder Back & Forth    3557   2.0       27  3.518519
65133  Blackadder Back & Forth    3576   3.0       27  3.518519
65133  Blackadder Back & Forth    3578   4.0       27  3.518519
65133  Blackadder Back & Forth    3608   3.0       27  3.518519

[1462309 rows x 5 columns]

In [33]:
df_correlationNumrateAvgrate.drop(['UserID', 'Rate'], inplace = True, axis = 1)
df_correlationNumrateAvgrate.drop_duplicates(inplace = True)
df_correlationNumrateAvgrate

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                             Title  NumRate   AvgRate
ID                                                   
1                        Toy Story       22  5.000000
2                          Jumanji       20  3.200000
3                 Grumpier Old Men       33  3.939394
4                Waiting to Exhale       38  4.026316
5      Father of the Bride Part II       87  3.850575
...                            ...      ...       ...
65088              Bedtime Stories       30  3.400000
65091          Manhattan Melodrama       53  3.820755
65126                        Choke       27  4.296296
65130           Revolutionary Road       34  4.058824
65133      Blackadder Back & Forth       27  3.518519

[10404 rows x 3 columns]

In [34]:
df_correlationNumrateAvgrate.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Title       object
NumRate      int64
AvgRate    float64
dtype: object

In [35]:
df_correlationNumrateAvgrate.corr(method = 'pearson')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          NumRate   AvgRate
NumRate  1.000000 -0.158309
AvgRate -0.158309  1.000000

There is weak negative correlation between Number of rates and Average rate. Theoreticaly, if movie has more rates, the lower average rate is.

However, result of correlation -0.158309 is close to 0, so we could assume that there is no correlation between Number of rates and Average rate of a movie.

## Correlation between average rate and year

In [36]:
df_correlationAvgrateYear = df_moviesAndRates.reset_index()
df_correlationAvgrateYear.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       Title  ID  UserID  Rate
0  Toy Story   1     122   5.0
1  Toy Story   1     185   5.0

In [37]:
df_correlationAvgrateYear.drop(['Title', 'UserID'], inplace = True, axis = 1)
df_correlationAvgrateYear.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   ID  Rate
0   1   5.0
1   1   5.0

In [38]:
df_correlationAvgrateYear = df_correlationAvgrateYear.groupby('ID').mean()
df_correlationAvgrateYear.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        Rate
ID          
1   5.000000
2   3.200000
3   3.939394
4   4.026316
5   3.850575

In [39]:
df_correlationAvgrateYear = df_correlationAvgrateYear.join(df_movieLens['Year']).reset_index()
df_correlationAvgrateYear.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   ID      Rate  Year
0   1  5.000000  1995
1   2  3.200000  1995
2   3  3.939394  1995
3   4  4.026316  1995
4   5  3.850575  1995

In [40]:
print(df_correlationAvgrateYear.corr(method = 'pearson'))
print(df_correlationAvgrateYear.head())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            ID      Rate      Year
ID    1.000000 -0.008909  0.198439
Rate -0.008909  1.000000  0.011901
Year  0.198439  0.011901  1.000000
   ID      Rate  Year
0   1  5.000000  1995
1   2  3.200000  1995
2   3  3.939394  1995
3   4  4.026316  1995
4   5  3.850575  1995

As you can see, there is weak positive correlation between Average Rate and Release Year.
Theoreticaly, if movie was released later (eg. 2005 vs 2010) the better average rate is.

However, result of correlation _0.011901_ is very close to _0_, so we simply just say that there is no correlation between Average Rate and Relese Year of a movie.